In [26]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk
import spacy
import pandas as pd 
import numpy as np

In [27]:
st = StanfordNERTagger(r'C:\Users\Aaryan\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\classifiers\english.muc.7class.distsim.crf.ser.gz',
r'C:\Users\Aaryan\Downloads\stanford-ner-4.2.0\stanford-ner-2020-11-17\stanford-ner.jar',
					   encoding='utf-8')

In [38]:
df = pd.read_csv('NER_Dataset.csv')
data = df.head(1000)

In [39]:
sentences =  data['Word'].apply(eval).apply(' '.join)

In [40]:
stanfordner_listoflists = []
#This was too slow to run on the entire dataset, 100 entries took about 2 minutes while 1000 entries took about 20 minutes. Hence, I assumed that 50000 entries would take about 16 hours to run.
for sentence in sentences:
    tokenized_text = word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)
    stanfordner_listoflists.append(classified_text)
    

In [41]:
import itertools
stanfordner_list = list(itertools.chain(*stanfordner_listoflists))
stanfordner_list = [elem[1] for elem in stanfordner_list]
stanfordner_list

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'DATE',
 'O',
 'O',
 'O',
 'ORGANIZATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORGANIZATION',
 'O',
 'O',
 'PERSON',
 'PERSON',
 'O',
 'DATE',
 'O',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'PERSON',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCATION'

In [42]:
nt = data.loc[:,'Tag']
nt_listoflists = nt.apply(eval).tolist()
ner_true = nt_listoflists
ner_truemerged2 = list(itertools.chain(*ner_true))

In [43]:
ner_truemergedctr = pd.Series(ner_truemerged2).value_counts().to_dict()
ner_predmergedctr = pd.Series(stanfordner_list).value_counts().to_dict()

ner_truemergedctr['B-geo'] = ner_truemergedctr['B-gpe'] + ner_truemergedctr['B-geo']
del ner_truemergedctr['B-gpe']
print(ner_truemergedctr)
print(ner_predmergedctr)

{'O': 18657, 'B-geo': 1074, 'B-tim': 442, 'B-org': 433, 'I-per': 354, 'B-per': 348, 'I-org': 326, 'I-geo': 144, 'I-tim': 129, 'B-art': 20, 'I-art': 16, 'B-nat': 10, 'I-nat': 6, 'I-gpe': 5, 'B-eve': 3, 'I-eve': 2}
{'O': 19426, 'LOCATION': 906, 'ORGANIZATION': 536, 'PERSON': 514, 'DATE': 449, 'MONEY': 96, 'PERCENT': 44, 'TIME': 2}


In [44]:
# Second dictionary provided
second_dict = stanfordner_listoflists

#accoording to https://pythonprogramming.net/named-entity-recognition-stanford-ner-tagger/  "7 class model for recognizing locations, persons, organizations, times, money, percents, and dates"

# Mapping from second to original keys with only B- labels
# i dont see anything for money and percent 
reverse_mapping = {
    'DATE': 'B-tim',
    #'TIME': 'B-tim',
    'PERSON': 'B-per',
    'LOCATION': 'B-geo',
    'ORGANIZATION': 'B-org',
    'O': 'O'
    #'GSP': 'B-nat',
    #'FACILITY': 'B-art',
    #'WORK_OF_ART': 'B-art',
    #'EVENT': 'B-eve'
}

# New dictionary to hold converted values
converted_dict = {}

# Convert without summing as we're using only B- labels
for key, value in ner_predmergedctr.items():
    new_key = reverse_mapping.get(key)  # Get the new key from the mapping
    if new_key:  # If the key exists in the mapping
        converted_dict[new_key] = value

print(converted_dict)

{'O': 19426, 'B-geo': 906, 'B-org': 536, 'B-per': 514, 'B-tim': 449}


In [35]:
#This accuracy changes if we uncomment the below if statement and add the tags that were not counted in our library. We are considering only matches for tags in all the accuracies
diff = {}
total = 0
for i in ner_truemergedctr:
    if i in converted_dict:
        diff[i] = converted_dict[i] - ner_truemergedctr[i]
        total += abs(diff[i])
#    if i not in converted_dict:
#        total += ner_truemergedctr[i]

accuracy = 1 - total/len(ner_truemerged2)
accuracy

0.9620141342756183

In [36]:
print("confusion matrix \n matches: ", len(ner_truemerged2) - total, "\n total tags: ", len(ner_truemerged2), "\n accuracy: ", accuracy)

confusion matrix 
 matches:  2178 
 total tags:  2264 
 accuracy:  0.9620141342756183


In [37]:
print(pd.crosstab(pd.Series(ner_truemerged2), pd.Series(stanfordner_list), rownames=['True'], colnames=['Predicted'], margins=True))

Predicted  DATE  LOCATION  MONEY     O  ORGANIZATION  PERCENT  PERSON   All
True                                                                       
B-art         0         0      0     3             1        0       0     4
B-geo         1        31      0    19             3        0       1    55
B-gpe         0         1      0    21             0        0       0    22
B-org         0         7      0     6            12        0       2    27
B-per         0         2      0    28             1        0      18    49
B-tim        20         1      0    21             0        0       0    42
I-art         0         0      0     2             0        0       0     2
I-geo         1         7      0     7             0        0       0    15
I-org         0         2      0     6             8        0       0    16
I-per         0         1      0    15             2        0      28    46
I-tim         4         0      0     8             0        0       0    12
O           